# Race Dashboard



This document describes the requirements and design decisions we will adopt for the Race Dashboard for the Sanofi Asset Efficiency challenge.  This document is intended to provide a detailed explanation of what is to be included on the dashboard, required data sources, and assumptions being made in the design.  It will also show the steps for getting the data into the necessary format.

## Scope/objective

The objective is to be able to display the documented metrics and categories as best suited on a Dashboard presentation for Sanofi staff to be able to access.


## Metrics / categories
The metrics have been mapped into sectors to mimic different sectors of a race track.  The metrics are:
 
 
 
•     Race = 8 Laps = 8 Months   
•     Lap = Monthly Progress   
<br>
<br>

•     **Sector 1 = OEE Improvement   
•     Sector 2 = OEE Variability Improvement   
•     Sector 3 = Stoppage Reduction   
•     Sector 4 = Changeover Improvement**   
• _Sector 5 = Most effective OEE application   
• Sector 6 = Best Innovation   
• Sector 7 = Most consistent OEE improvement progress   
• Sector 8 = Collaboration   
• Sector 9 = Team Spirit_

 
 


| Change log |
|:----------:|    


| Date | Initials | Comments |
|------|:---------|:---------|
| 2021-06-23 | MC | in leaderboard, replace NaN values in laptime calc with the max laptime for that lap
| 2021-06-23 | MC | use race review dates for grouping data, rather than calendar months
| 2021-06-24 |JB | missing OEE_Diff figures should default to 'OEE %' - OEE start point, not just 'OEE %' value
| 2021-06-24 |JB | for sector one, multiply sum of OEE_Diff by -1.  Should have always been doing this.
|2021-06-25 | MC | in leaderboard, change prev_race_time calc to include all but last 2 cols, to handle new race cols as they arrive
|2021-06-28 | MC | correct dates in nominations spreadsheet, and merge on 1 row with a date within the review period
|2021-06-29 | MC | corrected nominations spreadsheet for Lisieux IWK - should be TR200 Packaging

In [515]:
import pandas as pd
import numpy as np
import datetime


# Viz libs
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

# display options
# pd.options.display.float_format = "{:.2f}".format


## Read file and cleanse

- QSWeeklyFigures.xlsx should contain the latest data from QS   
- QSDashboard.xlsx has a list of plants/sites taking part, with original target OEE  
- QSWeeklyUnplannedTechLoss.xles should contain the latest data from QS (my unplanned chart by line/week)

#### Cleaning required:   
- OEE % needs converting to numeric, coerce the nulls to nan values
- I think W53-2020 data is bobbings - causes dup indexes for date 2021-01-10 and don't need 2020 data anyway...so dropping it

In [516]:
# dir = "C:/Users/mark_/McLaren Technology Group/McLaren Accelerator - Sanofi - Sanofi/Data Analysis/"
# dir = "C:/Users/mark_/Sanofi/Sanofi x McLaren sharing - General/Race Dashboard data/"
dir = "C:/Users/mark_/Documents/McLaren2021/Sanofi/Race Dashboard data/"
output_dir = "C:/Users/mark_/Documents/McLaren2021/Sanofi/Race Dashboard data/"

# dir = 'C:/Users/james.blood/Documents/McLarenSanofi/McLarenSanofi/data/'

file = (dir + 'OEE.xlsx')
df_weekly = pd.read_excel(file)
file = (dir + 'QSDashboard.xlsx')
df_dash = pd.read_excel(file)
file = (dir + 'Unplanned_tech_loss.xlsx')
df_techloss = pd.read_excel(file)
file = (dir + 'changeover.xlsx')
df_changeover = pd.read_excel(file)
df_weekly = df_weekly.loc[df_weekly['Week'].str.contains('2021')]
df_techloss = df_techloss.loc[df_techloss['Week'].str.contains('2021')]
df_weekly['OEE %'] = pd.to_numeric(df_weekly['OEE %'], errors='coerce')
df_techloss.rename(columns={'Unplanned losses - %OEE':'Unplanned_tech_loss'}, inplace=True)
df_techloss['Unplanned_tech_loss'] = pd.to_numeric(df_techloss['Unplanned_tech_loss'], errors='coerce')
df_changeover.rename(columns={'Change over losses - %OEE':'Changeover'}, inplace=True)
df_changeover['Changeover'] = pd.to_numeric(df_changeover['Changeover'], errors='coerce')
# don't use their progress figure as it's a static val
# df_dash.rename(columns={'⇗ OEE% progress':'OEE% progress'}, inplace=True)

create a datetime from the week number

In [517]:
df_weekly['WeekOfYear'] = pd.to_numeric(df_weekly['Week'].str[1:3])
df_weekly['Year'] = pd.to_numeric(df_weekly['Week'].str[4:])
dates = df_weekly.Year*100+df_weekly.WeekOfYear
df_weekly['Date'] = pd.to_datetime(dates.astype(str) + '0', format='%Y%W%w')
# df_weekly.drop(columns=['Year','WeekOfYear'], inplace=True)
df_weekly.head()

,Week,Line,OEE %,WeekOfYear,Year,Date
11,W01-2021,C2 Packaging Line,0.168970,1,2021,2021-01-10
12,W01-2021,C9 Packaging Line,NaN,1,2021,2021-01-10
13,W01-2021,GAMMA1,0.406686,1,2021,2021-01-10
14,W01-2021,IMA C80/2,0.510044,1,2021,2021-01-10
15,W01-2021,L18 Packaging Line,0.173736,1,2021,2021-01-10


In [518]:
#merge the 2 dataframes to get the start OEE
df_weekly = df_weekly.merge(df_dash[['Plant','Line', 'OEE  Start point','OEE% Target (2022)']],on='Line')

In [519]:
df_weekly = df_weekly.merge(df_techloss[['Line', 'Week', 'Unplanned_tech_loss']],on=['Line','Week'])

In [520]:
df_weekly = df_weekly.merge(df_changeover[['Line','Week','Changeover']])

#### Start Changeover

Start changeover value isn't provided, so going to calc our own start point using the average changeover for each site in 2021 up to April 2021.  This needs to be done before we drop the early 2021 rows.

This is then merged into the df_weekly dataframe as a loose join.

In [524]:
start_changeover_calc = df_weekly[['Plant','Line','Changeover']][df_weekly['Date'] < '2021-04-30'].groupby(['Plant','Line']).mean().reset_index()
start_changeover_calc.rename(columns={'Changeover':'start_changeover'}, inplace=True)
df_weekly = df_weekly.merge(start_changeover_calc[['Line','start_changeover']])

In [525]:
start_changeover_calc

,Plant,Line,start_changeover
0,Frankfurt,AL5 Packaging 1,0.104883
1,Frankfurt,AL6,0.106977
2,Frankfurt,M18 Filling,0.247334
3,Frankfurt,M21 Filling,0.252377
4,Frankfurt,M22 Filling,0.197075
5,Lisieux,SUPPO Packaging Line,0.101747
6,Lisieux,TR200 Packaging Line,0.193981
7,Maisons-Alfort,C2 Packaging Line,0.051707
8,Maisons-Alfort,C9 Packaging Line,0.016374
9,SCOPPITO,GAMMA1,0.245149


#### Dates for the Asset Challenge

Start Date is going to be fixed as 2021-04-01. Remove all the rows from df_weekly before this date

End Date will move and act as a cutoff before each Race meeting

In [527]:
start_date = '2021-04-01'
df_weekly = df_weekly[df_weekly['Date'] > start_date].sort_values('Date')

# do we need this?  We now have race review dates
# end_date = '2021-07-15'
# df_weekly = df_weekly[df_weekly['Date'] < end_date].sort_values('Date')

### PCT_CHANGE
Using pct_change python function with periods=4, giving a 4 week (4 previous rows) rolling pct_change figure
- I believe we are doing this rolling average calculation within Tableau at the moment, so this isn't being used here

- Not sure whether this is required for all of the categories?

In [528]:
df_weekly.sort_values(['Line','Date'], inplace = True)
df_weekly['OEE_pct_chg'] = (df_weekly.groupby('Line')['OEE %']
                                   .apply(pd.Series.pct_change, periods=4))
df_weekly['techloss_pct_chg'] = (df_weekly.groupby('Line')['Unplanned_tech_loss']
                                   .apply(pd.Series.pct_change, periods=4))
df_weekly['Changeover_pct_chg'] = (df_weekly.groupby('Line')['Changeover']
                                   .apply(pd.Series.pct_change, periods=4))
df_weekly.head()

,Week,Line,OEE %,WeekOfYear,Year,Date,Plant,OEE Start point,OEE% Target (2022),Unplanned_tech_loss,Changeover,start_changeover,OEE_pct_chg,techloss_pct_chg,Changeover_pct_chg
286,W13-2021,AL5 Packaging 1,NaN,13,2021,2021-04-04,Frankfurt,0.479693,0.5,NaN,NaN,0.104883,NaN,NaN,NaN
287,W14-2021,AL5 Packaging 1,NaN,14,2021,2021-04-11,Frankfurt,0.479693,0.5,NaN,NaN,0.104883,NaN,NaN,NaN
288,W15-2021,AL5 Packaging 1,0.449745,15,2021,2021-04-18,Frankfurt,0.479693,0.5,0.314964,0.087039,0.104883,NaN,NaN,NaN
289,W16-2021,AL5 Packaging 1,0.642652,16,2021,2021-04-25,Frankfurt,0.479693,0.5,0.135051,0.122727,0.104883,NaN,NaN,NaN
290,W17-2021,AL5 Packaging 1,0.505804,17,2021,2021-05-02,Frankfurt,0.479693,0.5,0.280638,0.063261,0.104883,NaN,NaN,NaN


## Standard Deviation
Calculate std_dev and mean on a 4 week rolling basis

Standard deviation is the square root of the variance, so no need to calculate both and have left var out

In [529]:
df_weekly['rolling_std'] = df_weekly.groupby('Line')['OEE %'].apply(lambda x : x.rolling(4,1).agg(np.std))
df_weekly.head(5)

,Week,Line,OEE %,WeekOfYear,Year,Date,Plant,OEE Start point,OEE% Target (2022),Unplanned_tech_loss,Changeover,start_changeover,OEE_pct_chg,techloss_pct_chg,Changeover_pct_chg,rolling_std
286,W13-2021,AL5 Packaging 1,NaN,13,2021,2021-04-04,Frankfurt,0.479693,0.5,NaN,NaN,0.104883,NaN,NaN,NaN,NaN
287,W14-2021,AL5 Packaging 1,NaN,14,2021,2021-04-11,Frankfurt,0.479693,0.5,NaN,NaN,0.104883,NaN,NaN,NaN,NaN
288,W15-2021,AL5 Packaging 1,0.449745,15,2021,2021-04-18,Frankfurt,0.479693,0.5,0.314964,0.087039,0.104883,NaN,NaN,NaN,NaN
289,W16-2021,AL5 Packaging 1,0.642652,16,2021,2021-04-25,Frankfurt,0.479693,0.5,0.135051,0.122727,0.104883,NaN,NaN,NaN,0.136405
290,W17-2021,AL5 Packaging 1,0.505804,17,2021,2021-05-02,Frankfurt,0.479693,0.5,0.280638,0.063261,0.104883,NaN,NaN,NaN,0.099233


In [530]:
file = (dir + 'Nominations Category Scoring.xlsx')
df_nom_sectors = pd.read_excel(file, sheet_name='Nomination scoring', usecols="A:H", parse_dates=['Date'])

In [531]:
df_nom_sectors['Date'] = pd.Series(df_nom_sectors['Date']).fillna(method='ffill')
df_nom_sectors = df_nom_sectors.fillna(0)

df_weekly = df_weekly.merge(df_nom_sectors[['Line','Plant','Date','Best Solution','Best Innovation','Improvement Iterations','Lessons and Sharing','Team Contribution and Spirit']], how='outer', on=['Date','Plant','Line'])
df_weekly

,Week,Line,OEE %,WeekOfYear,Year,Date,Plant,OEE Start point,OEE% Target (2022),Unplanned_tech_loss,...,start_changeover,OEE_pct_chg,techloss_pct_chg,Changeover_pct_chg,rolling_std,Best Solution,Best Innovation,Improvement Iterations,Lessons and Sharing,Team Contribution and Spirit
0,W13-2021,AL5 Packaging 1,NaN,13.0,2021.0,2021-04-04,Frankfurt,0.479693,0.5,NaN,...,0.104883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,W14-2021,AL5 Packaging 1,NaN,14.0,2021.0,2021-04-11,Frankfurt,0.479693,0.5,NaN,...,0.104883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,W15-2021,AL5 Packaging 1,0.449745,15.0,2021.0,2021-04-18,Frankfurt,0.479693,0.5,0.314964,...,0.104883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,W16-2021,AL5 Packaging 1,0.642652,16.0,2021.0,2021-04-25,Frankfurt,0.479693,0.5,0.135051,...,0.104883,NaN,NaN,NaN,0.136405,NaN,NaN,NaN,NaN,NaN
4,W17-2021,AL5 Packaging 1,0.505804,17.0,2021.0,2021-05-02,Frankfurt,0.479693,0.5,0.280638,...,0.104883,NaN,NaN,NaN,0.099233,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,NaN,AL5 Packaging 1,NaN,NaN,NaN,2021-09-12,Frankfurt,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
214,NaN,AL6,NaN,NaN,NaN,2021-09-12,Frankfurt,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
215,NaN,M18 Filling,NaN,NaN,NaN,2021-09-12,Frankfurt,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
216,NaN,M21 Filling,NaN,NaN,NaN,2021-09-12,Frankfurt,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0


## Create review dates
create a review_date column for grouping the data later, so we only get the data we're interested in for each review

In [533]:
def thurs_of_weekbefore(year, week):
    return datetime.date.fromisocalendar(year, week-1, 4)  # (year, week before (w-1), thursday)

review_weeks = [16, 20, 24, 28, 34, 38, 42, 47]
review_dates = []

for i in review_weeks:
    if i > 0:
        review_dates.append((thurs_of_weekbefore(2021,i)))

df_review_dates = pd.DataFrame(review_dates)
df_review_dates.rename(columns={0:'Review_Date'}, inplace=True)
df_review_dates['Review_Date'] = pd.to_datetime(df_review_dates.Review_Date)

# df_review_dates.info()
df_weekly = pd.merge_asof(df_weekly.sort_values('Date'), df_review_dates, left_on='Date', right_on='Review_Date', allow_exact_matches=True, direction='forward')


### Calculating Sector times


The lap time is a sum of the calculated sector scores + the pole position time from the F1 race data (eg for Paul Ricard it was 88 secs):



Sector [1-4] calculations   
**sector 1**
How much has your OEE increased / decreased?  Sum difference between each week and multiply total by -1.  This provides a negative figure to subtract from your laptime, so that larger OEE increase is rewarded with a bigger reduction in laptime

df_weekly['sector_1'] = df_weekly['OEE_Diff'].mul(-1)

OEE_Diff calculation
- Sort values by Line and Date
- Find the difference between each weekly OEE figure
- Fill NaN values from missing OEE figures with the weekly OEE minus OEE Start Point for that site


**Sector 2** 
How big was your rolling std deviation this period, over the previous 4 weeks std dev?  

df_weekly['sector_2'] = df_weekly['rolling_std']

rolling_std = rolling std deviation for past 4 weeks for each site


 
**Sector 3**
We want to reduce Unplanned tech loss (recorded as % of OEE) Unplanned tech loss is calculated within QlikSense but missing values sometimes.  Fill the missing values and then display the average Unplanned tech loss :

df_weekly['sector_3'] = df_weekly['Unplanned_tech_loss']


Populate missing unplanned tech loss:
- Create weekly min/max cols for Unplanned tech loss from any site 
- Merge those columns into df_weekly 
- fill any NaN unplanned tech loss rows with the max OEE calc'd for that week (bigger is worse)

 
**Sector 4**
We're trying to reduce changeover time (recorded as % of OEE).  
Start changeover value isn't provided, so calc our own start point for each Line using the average changeover in 2021 up to 30 April, 2021.
   
start_changeover_calc = df_weekly[['Plant','Line','Changeover']][df_weekly['Date'] < '2021-04-30'].groupby(['Plant','Line']).mean().reset_index()
start_changeover_calc.rename(columns={'Changeover':'start_changeover'}, inplace=True)
df_weekly = df_weekly.merge(start_changeover_calc[['Line','start_changeover']])



df_weekly['sector_4'] = df_weekly['Changeover_rolling_mean']

Changeover_mean = df_weekly.sort_values(by=['Line', 'Date'])[['Line', 'Date', 'Changeover', 'start_changeover']]
Changeover_mean['Changeover_rolling_mean'] = Changeover_mean.groupby('Line')['Changeover'].apply(lambda x : x.rolling(4,1).mean())
df_weekly = df_weekly.merge(Changeover_mean[["Line","Date","Changeover_rolling_mean"]], on=(["Line","Date"]))



**Clean the sectors of NaN before summing them**   
Sometimes, when we haven't got enough information for pct_change calcs, we were getting no values coming through for the lap_time.  We should make sure there is a value in each of the sectors, otherwise there is an unfair advantage by not having data available.  Find all NaN values and replace with the mean for that column(sector)

**Sectors 5 - 9**   
These scores are taken from the Nomination process.  Read in the Nomination s/s, merge any values we find with df_weekly, replace all NaN (missing) values with 0, and reduce the scores we find to 10% of their original value.  This value is then subtracted from the lap_time - so the better you do in the nominations the more your lap_time gets reduced by.

 
**lap_time**
df_weekly['lap_time'] = df_weekly[['sector_1','sector_2','sector_3','sector_4']].sum(axis=1)

#### Populate missing OEE %
- Find the weekly min/max OEE % from any site   
- Merge those columns into df_weekly   
- fill any NaN with the min OEE we calc'd for that week   

In [534]:
df_weekly_minmax = (df_weekly.assign(Data_Value=df_weekly['OEE %'].abs())
       .groupby(pd.Grouper(key='Date',freq='W'))['OEE %'].agg([('Min' , 'min'), ('Max', 'max')])
       .add_prefix('Week'))
df_weekly_minmax.reset_index(inplace=True)
df_weekly = df_weekly.merge(df_weekly_minmax[['Date','WeekMin','WeekMax']])
df_weekly['OEE %'].fillna(df_weekly.WeekMin, inplace=True)
df_weekly

,Week,Line,OEE %,WeekOfYear,Year,Date,Plant,OEE Start point,OEE% Target (2022),Unplanned_tech_loss,...,Changeover_pct_chg,rolling_std,Best Solution,Best Innovation,Improvement Iterations,Lessons and Sharing,Team Contribution and Spirit,Review_Date,WeekMin,WeekMax
0,W13-2021,AL5 Packaging 1,0.046124,13.0,2021.0,2021-04-04,Frankfurt,0.479693,0.500,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-15,0.046124,0.649336
1,W13-2021,M21 Filling,0.046124,13.0,2021.0,2021-04-04,Frankfurt,0.599671,0.650,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-15,0.046124,0.649336
2,W13-2021,M18 Filling,0.046124,13.0,2021.0,2021-04-04,Frankfurt,0.443522,0.650,0.114818,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-15,0.046124,0.649336
3,W13-2021,L25 Packaging Line,0.393740,13.0,2021.0,2021-04-04,Tours,0.351564,0.478,0.206618,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-15,0.046124,0.649336
4,W13-2021,SUPPO Packaging Line,0.432432,13.0,2021.0,2021-04-04,Lisieux,0.353021,0.530,0.148267,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-15,0.046124,0.649336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,NaN,L25 Packaging Line,NaN,NaN,NaN,2021-09-12,Tours,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,2021-09-16,NaN,NaN
214,NaN,L18 Packaging Line,NaN,NaN,NaN,2021-09-12,Tours,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,2021-09-16,NaN,NaN
215,NaN,M21 Filling,NaN,NaN,NaN,2021-09-12,Frankfurt,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,2021-09-16,NaN,NaN
216,NaN,C2 Packaging Line,NaN,NaN,NaN,2021-09-12,Maisons-Alfort,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,2021-09-16,NaN,NaN


#### We need the diff between the weekly OEE % figures, and the Weekly Changeover figures
Need something to calculate the OEE Progress and Changeover, otherwise we will have problems when we group and sum values later
- Create OEE_diff with OEE % from groupby of each Line, Week (only 1 row per week, so 'mean' will yield the same)   
- Find the diff between the rows in OEE_Diff for each Line   
- fillNA (first row for each Line) with OEE Start point - should only be needed on the first row for each Line   

repeat same logic for Changeover - there will be more NaN as start_changeover wasn't provided for all.  We populate this later

In [535]:
# this was calculating the wrong Diff - the first row of each site was looking at the previous site for all but the 1st calc
# needed to sort by Line and Date first 

# OEE_Diff = df_weekly.groupby(['Line',pd.Grouper(key='Date',freq='W')])['OEE %'].mean().reset_index()
# OEE_Diff["OEE_Diff"] = OEE_Diff["OEE %"].diff()
# df_weekly = df_weekly.merge(OEE_Diff[["Line","Date","OEE_Diff"]], on=(["Line","Date"]))

# df_weekly['OEE_Diff'].fillna(df_weekly['OEE %'] - df_weekly['OEE  Start point'], inplace=True)
# df_weekly[["Line","Date","OEE %","OEE_Diff"]].head(50).sort_values(by=['Line', 'Date'])

In [536]:
OEE_Diff = df_weekly.sort_values(by=['Line', 'Date'])[['Line','Date','OEE %','OEE  Start point']]
OEE_Diff['OEE_Diff'] = OEE_Diff.groupby('Line')['OEE %'].diff().fillna(df_weekly['OEE %'] - df_weekly['OEE  Start point'])
df_weekly = df_weekly.merge(OEE_Diff[["Line","Date","OEE_Diff"]], on=(["Line","Date"]))
df_weekly[["Line","Date","OEE %","OEE_Diff"]].head(50).sort_values(by=['Line', 'Date'])

,Line,Date,OEE %,OEE_Diff
0,AL5 Packaging 1,2021-04-04,0.046124,-0.433569
17,AL5 Packaging 1,2021-04-11,0.259071,0.212947
26,AL5 Packaging 1,2021-04-18,0.449745,0.190675
41,AL5 Packaging 1,2021-04-25,0.642652,0.192906
11,AL6,2021-04-04,0.367897,0.035240
23,AL6,2021-04-11,0.360681,-0.007216
36,AL6,2021-04-18,0.334140,-0.026541
47,AL6,2021-04-25,0.309545,-0.024595
10,C2 Packaging Line,2021-04-04,0.458414,0.060911
20,C2 Packaging Line,2021-04-11,0.530707,0.072293


In [538]:
# Changeover_Diff = df_weekly.groupby(['Line',pd.Grouper(key='Date',freq='W')])['Changeover'].mean().reset_index()
# Changeover_Diff["Changeover_Diff"] = Changeover_diff["Changeover"].diff()
# df_weekly = df_weekly.merge(Changeover_diff[["Line","Date","Changeover_Diff"]], on=(["Line","Date"]))

# df_weekly['Changeover_Diff'].fillna(df_weekly['start_changeover'] - df_weekly['Changeover'], inplace=True)

In [539]:
Changeover_Diff = df_weekly.sort_values(by=['Line', 'Date'])[['Line','Date','Changeover','start_changeover']]
Changeover_Diff['Changeover_Diff'] = Changeover_Diff.groupby('Line')['Changeover'].diff().fillna(df_weekly['start_changeover'] - df_weekly['Changeover'])
df_weekly = df_weekly.merge(Changeover_Diff[["Line","Date","Changeover_Diff"]], on=(["Line","Date"]))

In [540]:
Changeover_mean = df_weekly.sort_values(by=['Line', 'Date'])[['Line','Date','Changeover','start_changeover']]
Changeover_mean['Changeover_rolling_mean'] = Changeover_mean.groupby('Line')['Changeover'].apply(lambda x : x.rolling(4,1).mean())
df_weekly = df_weekly.merge(Changeover_mean[["Line","Date","Changeover_rolling_mean"]], on=(["Line","Date"]))

#### Populate missing Unplanned Tech Loss

- Create weekly min/max cols for Unplanned tech loss from any site   
- Merge those columns into df_weekly   
- fill any NaN rows with the max OEE calc'd for that week   

**this might be flawed!!** 

In [541]:
df_weekly_minmax = (df_weekly.assign(Data_Value=df_weekly['Unplanned_tech_loss'].abs())
       .groupby(pd.Grouper(key='Date',freq='W'))['Unplanned_tech_loss'].agg([('Min' , 'min'), ('Max', 'max')])
       .add_prefix('WeekUTL'))
df_weekly_minmax.reset_index(inplace=True)
df_weekly = df_weekly.merge(df_weekly_minmax[['Date','WeekUTLMin','WeekUTLMax']])
df_weekly['Unplanned_tech_loss'].fillna(df_weekly.WeekUTLMax, inplace=True)
df_weekly

,Week,Line,OEE %,WeekOfYear,Year,Date,Plant,OEE Start point,OEE% Target (2022),Unplanned_tech_loss,...,Lessons and Sharing,Team Contribution and Spirit,Review_Date,WeekMin,WeekMax,OEE_Diff,Changeover_Diff,Changeover_rolling_mean,WeekUTLMin,WeekUTLMax
0,W13-2021,AL5 Packaging 1,0.046124,13.0,2021.0,2021-04-04,Frankfurt,0.479693,0.500,0.465009,...,NaN,NaN,2021-04-15,0.046124,0.649336,-0.433569,NaN,NaN,0.114818,0.465009
1,W13-2021,M21 Filling,0.046124,13.0,2021.0,2021-04-04,Frankfurt,0.599671,0.650,0.465009,...,NaN,NaN,2021-04-15,0.046124,0.649336,-0.553548,NaN,NaN,0.114818,0.465009
2,W13-2021,M18 Filling,0.046124,13.0,2021.0,2021-04-04,Frankfurt,0.443522,0.650,0.114818,...,NaN,NaN,2021-04-15,0.046124,0.649336,-0.397398,-0.542447,0.789781,0.114818,0.465009
3,W13-2021,L25 Packaging Line,0.393740,13.0,2021.0,2021-04-04,Tours,0.351564,0.478,0.206618,...,NaN,NaN,2021-04-15,0.046124,0.649336,0.042176,0.150089,0.132655,0.114818,0.465009
4,W13-2021,SUPPO Packaging Line,0.432432,13.0,2021.0,2021-04-04,Lisieux,0.353021,0.530,0.148267,...,NaN,NaN,2021-04-15,0.046124,0.649336,0.079411,-0.077059,0.178806,0.114818,0.465009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,NaN,L25 Packaging Line,NaN,NaN,NaN,2021-09-12,Tours,NaN,NaN,NaN,...,0.0,0.0,2021-09-16,NaN,NaN,NaN,NaN,0.178769,NaN,NaN
214,NaN,L18 Packaging Line,NaN,NaN,NaN,2021-09-12,Tours,NaN,NaN,NaN,...,0.0,0.0,2021-09-16,NaN,NaN,NaN,NaN,0.111640,NaN,NaN
215,NaN,M21 Filling,NaN,NaN,NaN,2021-09-12,Frankfurt,NaN,NaN,NaN,...,0.0,0.0,2021-09-16,NaN,NaN,NaN,NaN,0.220155,NaN,NaN
216,NaN,C2 Packaging Line,NaN,NaN,NaN,2021-09-12,Maisons-Alfort,NaN,NaN,NaN,...,0.0,0.0,2021-09-16,NaN,NaN,NaN,NaN,0.212684,NaN,NaN


#### Populate missing Changeover 

In [542]:
df_weekly_minmax = (df_weekly.assign(Data_Value=df_weekly['Changeover'].abs())
       .groupby(pd.Grouper(key='Date',freq='W'))['Changeover'].agg([('Min' , 'min'), ('Max', 'max')])
       .add_prefix('WeekChangeover'))
df_weekly_minmax.reset_index(inplace=True)
df_weekly = df_weekly.merge(df_weekly_minmax[['Date','WeekChangeoverMin','WeekChangeoverMax']])
df_weekly['Changeover'].fillna(df_weekly.WeekChangeoverMax, inplace=True)
df_weekly

,Week,Line,OEE %,WeekOfYear,Year,Date,Plant,OEE Start point,OEE% Target (2022),Unplanned_tech_loss,...,Review_Date,WeekMin,WeekMax,OEE_Diff,Changeover_Diff,Changeover_rolling_mean,WeekUTLMin,WeekUTLMax,WeekChangeoverMin,WeekChangeoverMax
0,W13-2021,AL5 Packaging 1,0.046124,13.0,2021.0,2021-04-04,Frankfurt,0.479693,0.500,0.465009,...,2021-04-15,0.046124,0.649336,-0.433569,NaN,NaN,0.114818,0.465009,0.035164,0.789781
1,W13-2021,M21 Filling,0.046124,13.0,2021.0,2021-04-04,Frankfurt,0.599671,0.650,0.465009,...,2021-04-15,0.046124,0.649336,-0.553548,NaN,NaN,0.114818,0.465009,0.035164,0.789781
2,W13-2021,M18 Filling,0.046124,13.0,2021.0,2021-04-04,Frankfurt,0.443522,0.650,0.114818,...,2021-04-15,0.046124,0.649336,-0.397398,-0.542447,0.789781,0.114818,0.465009,0.035164,0.789781
3,W13-2021,L25 Packaging Line,0.393740,13.0,2021.0,2021-04-04,Tours,0.351564,0.478,0.206618,...,2021-04-15,0.046124,0.649336,0.042176,0.150089,0.132655,0.114818,0.465009,0.035164,0.789781
4,W13-2021,SUPPO Packaging Line,0.432432,13.0,2021.0,2021-04-04,Lisieux,0.353021,0.530,0.148267,...,2021-04-15,0.046124,0.649336,0.079411,-0.077059,0.178806,0.114818,0.465009,0.035164,0.789781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,NaN,L25 Packaging Line,NaN,NaN,NaN,2021-09-12,Tours,NaN,NaN,NaN,...,2021-09-16,NaN,NaN,NaN,NaN,0.178769,NaN,NaN,NaN,NaN
214,NaN,L18 Packaging Line,NaN,NaN,NaN,2021-09-12,Tours,NaN,NaN,NaN,...,2021-09-16,NaN,NaN,NaN,NaN,0.111640,NaN,NaN,NaN,NaN
215,NaN,M21 Filling,NaN,NaN,NaN,2021-09-12,Frankfurt,NaN,NaN,NaN,...,2021-09-16,NaN,NaN,NaN,NaN,0.220155,NaN,NaN,NaN,NaN
216,NaN,C2 Packaging Line,NaN,NaN,NaN,2021-09-12,Maisons-Alfort,NaN,NaN,NaN,...,2021-09-16,NaN,NaN,NaN,NaN,0.212684,NaN,NaN,NaN,NaN


#### Populate missing start_changeover (NaN)
If we've got this far and you still don't have a start_changeover, then you're a new site and can have this week's Changeover value

These rows still have NaN Changeover values

In [543]:
df_weekly[['Line','Date','Changeover','start_changeover']][df_weekly['Changeover'].isna()]

,Line,Date,Changeover,start_changeover
188,M22 Filling,2021-08-15,NaN,NaN
189,M21 Filling,2021-08-15,NaN,NaN
190,M18 Filling,2021-08-15,NaN,NaN
191,AL6,2021-08-15,NaN,NaN
192,AL5 Packaging 1,2021-08-15,NaN,NaN
193,SUPPO Packaging Line,2021-08-15,NaN,NaN
194,TR200 Packaging Line,2021-08-15,NaN,NaN
195,GAMMA1,2021-08-15,NaN,NaN
196,C2 Packaging Line,2021-08-15,NaN,NaN
197,IMA C80/2,2021-08-15,NaN,NaN


In [544]:
df_weekly.start_changeover.fillna(df_weekly.Changeover, inplace=True)
df_weekly['OEE  Start point'].fillna(df_weekly['OEE %'], inplace=True)
df_weekly['OEE% Target (2022)'].fillna(0.65, inplace=True)

#### Sector times

In [545]:
# df_weekly['sector_1'] = (df_weekly['WeekMax'] - df_weekly['OEE %'])
# df_weekly['sector_1'] = (df_weekly['OEE  Start point'] - df_weekly['OEE %'])
df_weekly['sector_1'] = df_weekly['OEE_Diff'].mul(-1)
df_weekly['sector_2'] = df_weekly['rolling_std']
df_weekly['sector_3'] = df_weekly['Unplanned_tech_loss']
# df_weekly['sector_4'] = (df_weekly['start_changeover'] - df_weekly['Changeover'])
# df_weekly['sector_4'] = (df_weekly['Changeover'] - df_weekly['start_changeover'])
df_weekly['sector_4'] = df_weekly['Changeover_rolling_mean']
# take 10% of the sector5-9 scores 
df_weekly[['sector_5','sector_6','sector_7','sector_8','sector_9']] = df_weekly[['Best Solution','Best Innovation','Improvement Iterations','Lessons and Sharing','Team Contribution and Spirit']] * -0.1
df_weekly[['sector_5','sector_6','sector_7','sector_8','sector_9']] = df_weekly[['sector_5','sector_6','sector_7','sector_8','sector_9']].fillna(0)

# we'll use these in the absence of values for a sector
df_weekly[['sector_1','sector_2','sector_3','sector_4']] = df_weekly[['sector_1','sector_2','sector_3','sector_4']].fillna(df_weekly[['sector_1','sector_2','sector_3','sector_4']].mean())

#this will sum and handle the NaN
df_weekly['lap_time'] = df_weekly[['sector_1','sector_2','sector_3','sector_4','sector_5','sector_6','sector_7','sector_8','sector_9']].sum(axis=1)

# now add the pole['Laptime'] from fastf1 to the lap_time adjustment we've created
# just use 88 secs rather than playing with timedeltas for now
# df_weekly['lap_time'] = pole['LapTime'] + pd.to_timedelta(df_weekly['lap_time'], unit='S')
# df_weekly['lap_time'] = 88 + df_weekly['lap_time']
df_weekly.groupby(['Line', pd.Grouper(key='Date', freq='W')])['lap_time'].sum()
# print (df_weekly['sector_1_time'] , df_weekly['sector_2_time'] , df_weekly['sector_3_time'], df_weekly['sector_4_time'])

Line                  Date      
AL5 Packaging 1       2021-04-04    1.158450
                      2021-04-11    0.440969
                      2021-04-18    0.314179
                      2021-04-25    0.183433
                      2021-05-02    0.607727
                                      ...   
TR200 Packaging Line  2021-06-20    0.635328
                      2021-06-27    0.297245
                      2021-07-04    1.544698
                      2021-08-15    0.526818
                      2021-09-12    0.413974
Name: lap_time, Length: 218, dtype: float64

#### Write out df_weekly to excel

In [546]:
df_weekly.to_excel(output_dir + "df_weekly_with_calcs.xlsx")

#### Monthly Calcs

Repeat the process for a df_monthly spreadsheet.  We will use this for calculating the Leader board  
group df_weekly by review_date so we can get the right data for each review meeting

In [548]:
# df_monthly = df_weekly.groupby([pd.Grouper(key='Date',freq='M'),'Line'])[['start_changeover','OEE  Start point','OEE %','Unplanned_tech_loss','Changeover','rolling_std','techloss_pct_chg','Changeover_pct_chg']].mean().reset_index()
# df_monthly = df_weekly.groupby([pd.Grouper(key='Date',freq='M'),'Line']).lap_time.sum().reset_index()
df_monthly = df_weekly.groupby(['Review_Date','Line']).lap_time.sum().reset_index()
# change the name of review_date to save renaming all references to Date later
df_monthly = df_monthly.rename(columns={'Review_Date':'Date'})
df_monthly

,Date,Line,lap_time
0,2021-04-15,AL5 Packaging 1,1.599419
1,2021-04-15,AL6,1.143054
2,2021-04-15,C2 Packaging Line,0.957890
3,2021-04-15,C9 Packaging Line,1.026479
4,2021-04-15,GAMMA1,1.158770
...,...,...,...
83,2021-09-16,M21 Filling,0.595611
84,2021-09-16,M22 Filling,0.496276
85,2021-09-16,MEDISEAL PURAN,0.532477
86,2021-09-16,SUPPO Packaging Line,0.461410


In [549]:
# df_monthly_minmax = (df_weekly.assign(Data_Value=df_weekly['OEE %'].abs())
#        .groupby(pd.Grouper(key='Date',freq='M'))['OEE %'].agg([('Min' , 'min'), ('Max', 'max')])
#        .add_prefix('Month'))
# df_monthly_minmax.reset_index(inplace=True)
# df_monthly = df_monthly.merge(df_monthly_minmax[['Date','MonthMin','MonthMax']])
df_monthly['lap_time'] = df_monthly['lap_time'] + 88
df_monthly

,Date,Line,lap_time
0,2021-04-15,AL5 Packaging 1,89.599419
1,2021-04-15,AL6,89.143054
2,2021-04-15,C2 Packaging Line,88.957890
3,2021-04-15,C9 Packaging Line,89.026479
4,2021-04-15,GAMMA1,89.158770
...,...,...,...
83,2021-09-16,M21 Filling,88.595611
84,2021-09-16,M22 Filling,88.496276
85,2021-09-16,MEDISEAL PURAN,88.532477
86,2021-09-16,SUPPO Packaging Line,88.461410


### Leader board table

In [550]:
# filter using the end_date to stop picking up future dated nomination rows of zero I created when joining the s/s
pivot = df_monthly[df_monthly['Date'] < end_date].pivot(index='Line', columns='Date', values='lap_time')
pivot.reset_index(inplace=True)
# pivot creates NaN for rows with no monthly data for each race review data
# populate each NaN value with the max for that column - so they get the max laptime for that race
# we can search for cols [1:] and find all cols after Date and Line
pivot.iloc[:,1:] = pivot.iloc[:,1:].fillna(pivot.iloc[:,1:].max())

# sum all the columns to get a race_time
pivot['race_time'] = pivot.sum(axis=1)
# sum all but the last 2 cols (this lap and the race_time) to calc prev_race_time

# pivot['prev_race_time'] = pivot[pivot.columns[2]] + pivot[pivot.columns[3]]
pivot['prev_race_time'] = pivot.iloc[:,1:-2].sum(axis=1)

pivot = pivot.merge(df_dash[['Plant','Line']], on='Line')
pivot.sort_values('race_time', inplace=True)
pivot['position'] = np.arange(1,len(pivot) + 1)
pivot['gap_to_leader'] = pivot['race_time'] - pivot['race_time'].iloc[0]
pivot.sort_values('prev_race_time', inplace=True)
pivot['prev_position'] = np.arange(1,len(pivot) + 1)
pivot['Gain/Loss'] = pivot.prev_position - pivot.position
pivot.sort_values('race_time', inplace=True)
pivot['interval'] = pivot.race_time.diff()
pivot = pivot.merge(df_dash[['Line','OEE  Start point', '⇗ OEE% progress', 'OEE% Target (2022)']], on='Line')
pivot

,Line,2021-04-15 00:00:00,2021-05-13 00:00:00,2021-06-10 00:00:00,2021-07-08 00:00:00,race_time,prev_race_time,Plant,position,gap_to_leader,prev_position,Gain/Loss,interval,OEE Start point,⇗ OEE% progress,OEE% Target (2022)
0,GAMMA1,89.158770,69.325767,68.226756,90.038576,316.749870,226.711294,SCOPPITO,1,0.000000,1,0,NaN,0.418683,0.085148,0.570
1,IMA C80/2,88.915460,69.256522,75.232693,89.520411,322.925085,233.404675,SCOPPITO,2,6.175215,2,0,6.175215,0.451031,0.043365,0.580
2,L18 Packaging Line,88.852506,74.226704,89.407996,89.262767,341.749974,252.487206,Tours,3,25.000103,3,0,18.824888,0.377683,0.086173,0.547
3,L25 Packaging Line,88.854983,74.908842,90.011979,89.722251,343.498054,253.775803,Tours,4,26.748184,4,0,1.748081,0.351564,0.001613,0.478
4,M22 Filling,88.959190,81.806345,84.857670,89.844474,345.467680,255.623206,Frankfurt,5,28.717809,5,0,1.969625,0.530068,0.120280,0.650
5,M21 Filling,89.719397,83.496534,84.767181,90.769388,348.752500,257.983112,Frankfurt,6,32.002630,6,0,3.284821,0.599671,0.022006,0.650
6,M18 Filling,89.965673,82.596174,85.446232,90.866765,348.874844,258.008079,Frankfurt,7,32.124974,7,0,0.122344,0.443522,-0.010057,0.650
7,AL6,89.143054,90.323044,84.205598,89.779721,353.451417,263.671696,Frankfurt,8,36.701547,8,0,4.576573,0.332657,0.078541,0.450
8,LINE 01 - UHLMANN 1880,89.965673,88.720611,89.037542,89.028156,356.751981,267.723826,SUZANO,9,40.002111,10,1,3.300564,0.427854,0.015085,0.650
9,MEDISEAL PURAN,89.965673,88.720611,89.037542,89.028156,356.751981,267.723826,SUZANO,10,40.002111,11,1,0.000000,0.592206,-0.025165,0.650


#### write this out for tableau

In [551]:
pivot.to_csv(output_dir + "leaderboard.csv")

END OF PROCESSING - Sanity checks below

In [552]:
pivot.sort_values(pivot.columns[1], inplace=True)
pivot['apr_position'] = np.arange(1,len(pivot) + 1)
pivot['temp_race_time'] = pivot[pivot.columns[[1,2]]].sum(axis=1)
pivot.sort_values(by='temp_race_time', inplace=True)
pivot['may_position'] = np.arange(1,len(pivot) + 1)
pivot['temp_race_time'] = pivot[pivot.columns[[1,2,3]]].sum(axis=1)
pivot.sort_values(by='temp_race_time', inplace=True)
pivot['jun_position'] = np.arange(1,len(pivot) + 1)
pivot['temp_race_time'] = pivot[pivot.columns[[1,2,3,4]]].sum(axis=1)
pivot.sort_values(by='temp_race_time', inplace=True)
pivot['jly_position'] = np.arange(1,len(pivot) + 1)

In [553]:
# regex filter on column name contains Line, position or 2021
pivot.filter(regex='Line|position|2021')

,Line,2021-04-15 00:00:00,2021-05-13 00:00:00,2021-06-10 00:00:00,2021-07-08 00:00:00,position,prev_position,apr_position,may_position,jun_position,jly_position
0,GAMMA1,89.158770,69.325767,68.226756,90.038576,1,1,10,2,1,1
1,IMA C80/2,88.915460,69.256522,75.232693,89.520411,2,2,5,1,2,2
2,L18 Packaging Line,88.852506,74.226704,89.407996,89.262767,3,3,3,3,3,3
3,L25 Packaging Line,88.854983,74.908842,90.011979,89.722251,4,4,4,4,4,4
4,M22 Filling,88.959190,81.806345,84.857670,89.844474,5,5,7,5,5,5
5,M21 Filling,89.719397,83.496534,84.767181,90.769388,6,6,12,7,6,6
6,M18 Filling,89.965673,82.596174,85.446232,90.866765,7,7,13,6,7,7
7,AL6,89.143054,90.323044,84.205598,89.779721,8,8,9,15,8,8
8,LINE 01 - UHLMANN 1880,89.965673,88.720611,89.037542,89.028156,9,10,14,10,10,9
9,MEDISEAL PURAN,89.965673,88.720611,89.037542,89.028156,10,11,15,11,11,10


In [554]:
# df_weekly.groupby(['Review_Date','Line']).lap_time.sum().reset_index()
sectors = df_weekly.filter(regex='Line|sector|Review_Date')

In [555]:
# sectors[sectors.columns[[2,3,4,5]]].sum(axis=1)
sectors = df_weekly.filter(regex='Line|sector|Review_Date')
sectors = sectors.iloc[:,0:6]
sectors = sectors.groupby(['Line','Review_Date']).sum().reset_index()
sectors['time'] = sectors.sum(axis=1)
sectors_pivot = sectors.pivot(index='Line', columns='Review_Date', values='time')
sectors_pivot.reset_index(inplace=True)
sectors_pivot
# sectors[sectors[sectors.columns[[2,3,4,5]]].sum(axis=1)
# sectors[sectors.Line.str.contains('AL5')]

Review_Date,Line,2021-04-15 00:00:00,2021-05-13 00:00:00,2021-06-10 00:00:00,2021-07-08 00:00:00,2021-08-19 00:00:00,2021-09-16 00:00:00
0,AL5 Packaging 1,1.599419,1.653462,2.618101,2.426822,0.455217,0.498887
1,AL6,1.143054,2.323044,1.805598,1.779721,0.418774,0.435521
2,C2 Packaging Line,0.957890,2.451874,2.261138,2.572581,0.539264,0.588141
3,C9 Packaging Line,1.026479,1.747841,2.361488,1.831886,0.444573,0.429521
4,GAMMA1,1.158770,1.725767,1.926756,2.038576,0.611908,0.651116
5,IMA C80/2,0.915460,1.656522,1.732693,1.520411,0.545559,0.545278
6,L18 Packaging Line,0.852506,1.426704,1.407996,1.262767,0.502970,0.487097
7,L25 Packaging Line,0.854983,2.108842,2.011979,1.722251,0.591038,0.554225
8,LINE 01 - UHLMANN 1880,NaN,0.720611,1.037542,1.028156,0.532477,0.532477
9,M18 Filling,1.965673,2.596174,2.646232,2.866765,0.764240,0.939725


In [556]:
# search for cols [1:] and find all cols after Date and Line
sectors_pivot.iloc[:,1:] = sectors_pivot.iloc[:,1:].fillna(sectors_pivot.iloc[:,1:].max())


In [557]:
sectors_pivot.sort_values(sectors_pivot.columns[1], inplace=True)
sectors_pivot['apr_position'] = np.arange(1,len(sectors_pivot) + 1)
sectors_pivot['temp_race_time'] = sectors_pivot[sectors_pivot.columns[[1,2]]].sum(axis=1)
sectors_pivot.sort_values(by='temp_race_time', inplace=True)
sectors_pivot['may_position'] = np.arange(1,len(sectors_pivot) + 1)
sectors_pivot['temp_race_time'] = sectors_pivot[sectors_pivot.columns[[1,2,3]]].sum(axis=1)
sectors_pivot.sort_values(by='temp_race_time', inplace=True)
sectors_pivot['jun_position'] = np.arange(1,len(sectors_pivot) + 1)
sectors_pivot['temp_race_time'] = sectors_pivot[sectors_pivot.columns[[1,2,3,4]]].sum(axis=1)
sectors_pivot.sort_values(by='temp_race_time', inplace=True)
sectors_pivot['jly_position'] = np.arange(1,len(sectors_pivot) + 1)
sectors_pivot

Review_Date,Line,2021-04-15 00:00:00,2021-05-13 00:00:00,2021-06-10 00:00:00,2021-07-08 00:00:00,2021-08-19 00:00:00,2021-09-16 00:00:00,apr_position,temp_race_time,may_position,jun_position,jly_position
8,LINE 01 - UHLMANN 1880,1.965673,0.720611,1.037542,1.028156,0.532477,0.532477,13,4.751981,5,3,1
12,MEDISEAL PURAN,1.965673,0.720611,1.037542,1.028156,0.532477,0.532477,15,4.751981,6,4,2
6,L18 Packaging Line,0.852506,1.426704,1.407996,1.262767,0.502970,0.487097,3,4.949974,2,1,3
5,IMA C80/2,0.915460,1.656522,1.732693,1.520411,0.545559,0.545278,5,5.825085,4,6,4
13,SUPPO Packaging Line,0.590586,1.612821,1.992146,1.679336,0.443389,0.461410,1,5.874889,1,5,5
11,M22 Filling,0.959190,1.806345,2.057670,1.844474,0.577587,0.496276,7,6.667680,7,8,6
7,L25 Packaging Line,0.854983,2.108842,2.011979,1.722251,0.591038,0.554225,4,6.698054,10,9,7
4,GAMMA1,1.158770,1.725767,1.926756,2.038576,0.611908,0.651116,10,6.849870,9,7,8
14,TR200 Packaging Line,0.715615,1.591164,1.381941,3.181492,0.526818,0.413974,2,6.870212,3,2,9
3,C9 Packaging Line,1.026479,1.747841,2.361488,1.831886,0.444573,0.429521,8,6.967695,8,10,10


In [560]:
df_nom_sectors.Line.unique()

array(['L18 Packaging Line', 'L25 Packaging Line',
       'LINE 01 - UHLMANN 1880', 'MEDISEAL PURAN', 'GAMMA1', 'IMA C80/2',
       'C2 Packaging Line', 'C9 Packaging Line', 'TR200 Packaging Line',
       'SUPPO Packaging Line', 'AL5 Packaging 1', 'AL6', 'M18 Filling',
       'M21 Filling', 'M22 Filling'], dtype=object)